### 函数的工作原理

In [3]:
import dis

In [1]:
def foo():
    bar()
    
def bar():
    pass

py代码是运行在C程序之上：  
PyEval_EvalFramEx(C函数)去执行foo

In [4]:
dis.dis(foo)

  2           0 LOAD_GLOBAL              0 (bar)
              2 CALL_FUNCTION            0
              4 POP_TOP
              6 LOAD_CONST               0 (None)
              8 RETURN_VALUE


python一切皆对象：  
使用dis函数显示【栈帧对象】-->【字节码对象】

当foo调用子函数bar，又会创建一个栈帧  
所有的栈帧都是分配在堆内存上（不释放会一直存在）  
所以：栈帧可以独立于调用中存在

In [5]:
import inspect
frame=None

In [6]:
def foo():
    bar()
    
def bar():
    global frame
    frame=inspect.currentframe()

In [8]:
foo()

In [9]:
# 提取bar的栈帧
frame.f_code.co_name

'bar'

In [11]:
# 提取foo的栈帧
caller_frame=frame.f_back
caller_frame.f_code.co_name

'foo'

生成器对象就是利用了python栈帧都是分配在堆内存上这一特性

In [12]:
def gen_func():
    yield 1
    name="user"
    yield 2
    age=30
    return "hehe"

In [14]:
gen=gen_func()

In [16]:
dis.dis(gen)

  2           0 LOAD_CONST               1 (1)
              2 YIELD_VALUE
              4 POP_TOP

  3           6 LOAD_CONST               2 ('user')
              8 STORE_FAST               0 (name)

  4          10 LOAD_CONST               3 (2)
             12 YIELD_VALUE
             14 POP_TOP

  5          16 LOAD_CONST               4 (30)
             18 STORE_FAST               1 (age)

  6          20 LOAD_CONST               5 ('hehe')
             22 RETURN_VALUE


### 总结：  
这样整个生成器对象就存在与堆内存中，可以独立存在，每次执行一次函数，就会生成一个栈帧对象。  
我们可以在任何地方，只要能拿到这个栈帧对象就能够往前走。这也是python中协程的一个理论基础。

### 生成器运用

In [27]:
from collections import UserList

由于内部list是用C写的，可以使用coll中的list

In [43]:
#500G的数据，只有一行

def myreadlines(f,newline):
    buf=""
    while True:
        
        while newline in buf:
            pos=buf.index(newline)
            yield buf[:pos]
            buf=buf[pos+len(newline):]
            
        chunk=f.read(4096)
        
        if not chunk:
            yield buf
            break
            
        buf += chunk

假设分隔符不在buf中：读取文件中的一段数据【长度自己定义】  
存在了分隔符的数据：1，找到位置；2，返回切分完的字符串；3，保留截取之后的字符串  
存在分隔符的数据处理完毕：1，接着读文件中的数据；2，截取数据合并上读取出来的数据；3，重复第二步  
读到最后没有了数据了：直接返回【截取数据合并上读取出来的数据】buf

In [29]:
with open("b.txt") as f:
    for line in myreadlines(f,","):
        print(line)

sddsdasdsdsdasdasd
adsiadhishdiashdisdhis
dhasdiashdiasd
vasudsadisabxasbadsbdsd
bdusbdasdiashdisnidas
dsbdbsdbia



In [42]:
# def myreadlines2(f,newline):
#     buf=""
#     while True:
#         while newline in buf:
#             pos=buf.index(newline)
# #             print(pos)
#             yield buf[:pos]
#             buf=buf[pos+len(newline):]
#         chunk=f.read(43)
# #         print(chunk)
#         if not chunk:
#             yield buf
#             break
#         buf+=chunk
#         print("|"+buf+"|")
        
# with open("b.txt") as f:
#     for line in myreadlines2(f,","):
#         pass
# #         print(line)
